In [9]:
import pandas as pd
import os
import pickle

In [10]:
data = pd.read_csv('./data/time_series_covid19_confirmed_global.csv')
c = data.groupby(data.columns[1]).sum()
df = c.drop(['Lat', 'Long', 'Province/State'], axis=1)

def first_non_zero_index(row):
    non_zero_indices = row.to_numpy().nonzero()[0]
    return non_zero_indices[0] if len(non_zero_indices) > 0 else -1

first_infected = df.apply(first_non_zero_index, axis=1).nsmallest(1).index
print(first_infected.values)

['China']


In [11]:
airports_df = pd.read_csv('./data/airports.csv', sep=',', engine='python')
infected_airports = set()
for infected in first_infected:
    infected_airports.update(set(airports_df[airports_df['Country'] == infected]['IATA']))
infected_airports.remove('\\N')


In [12]:
import networkx as nx
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
from plotly.express import colors
from plotly.colors import sample_colorscale

G = nx.DiGraph()
t  = airports_df.drop_duplicates(subset='Country', keep='first')
for _, row in t.iterrows():
    G.add_node(row['Country'], Lat = row['Latitude'], Long = row['Longitude'])

routes_df = pd.read_csv('./data/routes.csv', sep=',', engine='python')

In [13]:

infecting_countries = set()
# for infected airport, connect country that has flight from infected airport
for infected_airport in infected_airports:
    source_ctry = airports_df[airports_df['IATA'] == infected_airport]['Country'].iloc[0]
    infecting_countries.add(source_ctry)
    potentially_infected = routes_df[routes_df['Source airport'] == infected_airport]['Destination airport']
    potentialy_infected_countries = set()
    for candidate in potentially_infected:
        if len(airports_df[airports_df['IATA'] == candidate]['Country']):
            potentialy_infected_countries.add(airports_df[airports_df['IATA'] == candidate]['Country'].iloc[0])
        
    for c in potentialy_infected_countries:
        if c != source_ctry:
            G.add_edge(source_ctry, c, source_ctry=source_ctry)
        

In [14]:

num_colors = len(infecting_countries)
color_scale = sample_colorscale('Viridis', [n / num_colors for n in range(num_colors)])

# Assign colors to each group (first case date)
color_map = {ctry: color_scale[i % num_colors] for i, ctry in enumerate(infecting_countries)}
    
for node in G.nodes:
    try:
        G.nodes[node]
    except KeyError:
        pass
    
node_trace = go.Scattergeo(
    lon=[G.nodes[node]['Long'] for node in G.nodes],
    lat=[G.nodes[node]['Lat'] for node in G.nodes],
    # text=[f"{node}<br>First Case: {G.nodes[node]['first_case_date']}<br>Province or state: {G.nodes[node]['province_state']}" for node in G.nodes], #.date()
    mode='markers',
    marker=dict(
        size=[1 for node in G.nodes],
        color=[color_map[node] for node in infecting_countries],
        opacity=0.7,
        colorscale='Viridis',
        colorbar=dict(title='First Case Date')
    )
)

In [15]:

# Create edge traces
edge_traces = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['Long'], G.nodes[edge[0]]['Lat']
    x1, y1 = G.nodes[edge[1]]['Long'], G.nodes[edge[1]]['Lat']
    #print(edge('date'))
    edge_color = color_map[edge[0]]
    edge_trace = go.Scattergeo(
        lon=[x0, x1, None],
        lat=[y0, y1, None],
        mode='lines',
        line=dict(width=0.5, color=edge_color),#dict(width=0.5, color='gray'),
        opacity=0.5
    )
    edge_traces.append(edge_trace)


In [16]:

# Create the figure
fig = go.Figure(data=edge_traces + [node_trace])

fig.update_layout(
    title='COVID-19 First Case Connections on World Map',
    showlegend=False,
    geo=dict(
        scope='world',
        projection_type='equirectangular',
        showland=True,
        landcolor='rgb(217, 217, 217)',
        subunitwidth=1,
        countrywidth=1,
        subunitcolor="rgb(255, 255, 255)",
        countrycolor="rgb(255, 255, 255)"
    )
)

fig.show()